# Notes on Generalized Advantage Estimation




## How to compute GAE on your own?
(Note that for this reading you need to understand the calculation of [GAE](https://arxiv.org/abs/1506.02438) advantage first)

In terms of code implementation, perhaps the most difficult and annoying part is computing GAE advantage. Just now, we use the `self.compute_episodic_return()` method inherited from `BasePolicy` to save us from all those troubles. However, it is still important that we know the details behind this.

To compute GAE advantage, the usage of `self.compute_episodic_return()` may go like:

```python
batch, indices = dummy_buffer.sample(0)  # 0 means sampling all the data from the buffer
returns, advantage = Algorithm.compute_episodic_return(
    batch=batch,
    buffer=dummy_buffer,
    indices=indices,
    v_s_=np.zeros(10),
    v_s=np.zeros(10),
    gamma=1.0,
    gae_lambda=1.0,
)
```

In the code above, we sample all the 10 data in the buffer and try to compute the GAE advantage. However, the way the returns are computed here might be a bit misleading. In fact, the last episode is unfinished, but its last step saved in the batch is treated as a terminal state, since it assumes that there are no future rewards. The episode is not terminated yet, it is truncated, so the agent could still get rewards in the future. Terminated and truncated episodes should indeed be treated differently.
The return of a step is the (discounted) sum of the future rewards from that step until the end of the episode. 
\begin{equation}
R_{t}=\sum_{t}^{T} \gamma^{t} r_{t}
\end{equation}
Thus, at the last step of a terminated episode the return is equal to the reward at that state, since there are no future states.
\begin{equation}
R_{T,terminated}=r_{T}
\end{equation}

However, if the episode was truncated the return at the last step is usually better represented by the estimated value of that state, which is the expected return from that state onwards.
\begin{align*}
R_{T,truncated}=V^{\pi}\left(s_{T}\right)  \quad & \text{or} \quad R_{T,truncated}=Q^{\pi}(s_{T},a_{T})
\end{align*}
Moreover, if the next state was also observed (but not its reward), then an even better estimate would be the reward of the last step plus the discounted value of the next state.
\begin{align*}
R_{T,truncated}=r_T+\gamma V^{\pi}\left(s_{T+1}\right)
\end{align*}


As we know, we need to estimate the value function of every observation to compute GAE advantage. So in `v_s` is the value of `batch.obs`, and in `v_s_` is the value of `batch.obs_next`. This is usually computed by:

`v_s = critic(batch.obs)`,

`v_s_ = critic(batch.obs_next)`,

where both `v_s` and `v_s_` are 10 dimensional arrays and `critic` is usually a neural network.

After we've got all those values, GAE can be computed following the equation below.

\begin{aligned}
\hat{A}_{t}^{\mathrm{GAE}(\gamma, \lambda)}: =& \sum_{l=0}^{\infty}(\gamma \lambda)^{l} \delta_{t+l}^{V}
\end{aligned}

where

\begin{equation}
\delta_{t}^{V} \quad=-V\left(s_{t}\right)+r_{t}+\gamma V\left(s_{t+1}\right)
\end{equation}


Unfortunately, if you  follow this equation, which is taken from the paper, you probably will get a slightly lower performance than you expected. There are at least 3 "bugs" in this equation.

**First** is that Gym always returns you a `obs_next` even if this is already the last step. The value of this timestep is exactly 0 and you should not let the neural network estimate it.

```python
# Assume v_s_ is got by calling critic(batch.obs_next)
v_s_ = np.ones(10)
v_s_ *= ~batch.done
```

After the fix above, we will perhaps get a more accurate estimate.

**Secondly**, you must know when to stop bootstrapping. Usually we stop bootstrapping when we meet a `done` flag. However, in the buffer above, the last (10th) step is not marked by done=True, because the collecting has not finished. We must know all those unfinished steps so that we know when to stop bootstrapping.

Luckily, this can be done under the assistance of buffer because buffers in Tianshou not only store data, but also help you manage data trajectories.

```python
unfinished_indexes = dummy_buffer.unfinished_index()
done_indexes = np.where(batch.done)[0]
stop_bootstrap_ids = np.concatenate([unfinished_indexes, done_indexes])
```

**Thirdly**, there are some special indexes which are marked by done flag, however its value for obs_next should not be zero. It is again because done does not differentiate between terminated and truncated. These steps are usually those at the last step of an episode, but this episode stops not because the agent can no longer get any rewards (value=0), but because the episode is too long so we have to truncate it. These kind of steps are always marked with `info['TimeLimit.truncated']=True` in Gym.

As a result, we need to rewrite the equation above

`v_s_ *= ~batch.done`

to

```
mask = batch.info['TimeLimit.truncated'] | (~batch.done)
v_s_ *= mask

```





## Summary
If you already felt bored by now, simply remember that Tianshou can help handle all these little details so that you can focus on the algorithm itself. Just call `Algorithm.compute_episodic_return()`.

If you still feel interested, we would recommend you check Appendix C in this [paper](https://arxiv.org/abs/2107.14171v2) and implementation of `Algorithm.value_mask()` and `Algorithm.compute_episodic_return()` for details.

<center>
<img src=../_static/images/timelimit.svg></img>
</center>
<center>
<img src=../_static/images/policy_table.svg></img>
</center>